- exchange는 time_lag가 없음

- 고쳐야 하는 부분 : LSTM 적용 범위가 이상함

In [1]:
%pip install -q -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd

In [3]:
# 하이퍼 파라미터
input_size_ex = 24
input_size_comb = 24
hidden_size = 192
output_size = 1

batch_size = 16

lstm_hidden_dim = 192
lstm_n_layer = 1
bidirectional = False
fc_hidden_list = []
dropout_p = 0.0
batch_norm = False

In [4]:
import sys

# 경로 설정
src_path = os.path.abspath('../src')
print(src_path)
sys.path.append(src_path)

/Users/yoom618/Downloads/timelag_correlation_prediction/src


In [5]:
import dataset as data_module

In [6]:
start_date = '2024-01-01'
end_date = '2024-11-22'

input_days = 24
time_lag = 12

in_cols_ex = ['test_X']
in_cols_comb = ['test_X', 'test_Y']
out_cols = ['test_X']

scaling='standard'

train_ex = data_module.PriceDataset(
    start_date, end_date, 
    phase='train',
    in_columns=in_cols_ex,
    out_columns=out_cols,
    input_days=input_days,
    time_lag=time_lag,
    data_dir='../data',
    test_size=0,
    train_ratio=0.8,
    scaling=scaling)

test_ex = data_module.PriceDataset(
    start_date, end_date, 
    phase='valid',
    in_columns=in_cols_ex,
    out_columns=out_cols,
    input_days=input_days,
    time_lag=time_lag,
    data_dir='../data',
    test_size=0,
    train_ratio=0.8,
    scaling=scaling)

train_comb = data_module.PriceDataset(
    start_date, end_date, 
    phase='train',
    in_columns=in_cols_comb,
    out_columns=out_cols,
    input_days=input_days,
    time_lag=time_lag,
    data_dir='../data',
    test_size=0,
    train_ratio=0.8,
    scaling=scaling)

test_comb = data_module.PriceDataset(
    start_date, end_date, 
    phase='valid',
    in_columns=in_cols_comb,
    out_columns=out_cols,
    input_days=input_days,
    time_lag=time_lag,
    data_dir='../data',
    test_size=0,
    train_ratio=0.8,
    scaling=scaling)

Load Data from ../data/preprocessed/240101_241122_dGVzdA=='.pkl
	Data Period: 240101 - 241122
	Symbols: {'test'}
	-> Selected Data Period: 2024-01-01 - 2024-11-22

Load Data from ../data/preprocessed/240101_241122_dGVzdA=='.pkl
	Data Period: 240101 - 241122
	Symbols: {'test'}
	-> Selected Data Period: 2024-01-01 - 2024-11-22

Load Data from ../data/preprocessed/240101_241122_dGVzdA=='.pkl
	Data Period: 240101 - 241122
	Symbols: {'test'}
	-> Selected Data Period: 2024-01-01 - 2024-11-22

Load Data from ../data/preprocessed/240101_241122_dGVzdA=='.pkl
	Data Period: 240101 - 241122
	Symbols: {'test'}
	-> Selected Data Period: 2024-01-01 - 2024-11-22



In [7]:
print(train_ex.x.shape, train_ex.y.shape, len(train_ex.y_index))
print(test_ex.x.shape, test_ex.y.shape, len(test_ex.y_index))
print(train_comb.x.shape, train_comb.y.shape, len(train_comb.y_index))
print(test_comb.x.shape, test_comb.y.shape, len(test_comb.y_index))

torch.Size([156, 24, 1]) torch.Size([156, 1]) 156
torch.Size([39, 24, 1]) torch.Size([39, 1]) 39
torch.Size([156, 24, 2]) torch.Size([156, 1]) 156
torch.Size([39, 24, 2]) torch.Size([39, 1]) 39


In [8]:
from torch.utils.data import DataLoader

train_loader_ex = DataLoader(train_ex, batch_size=batch_size, shuffle=True)
test_loader_ex = DataLoader(test_ex, batch_size=batch_size, shuffle=False)

train_loader_comb = DataLoader(train_comb, batch_size=batch_size, shuffle=True)
test_loader_comb = DataLoader(test_comb, batch_size=batch_size, shuffle=False)

In [9]:
import models as model_module

model_ex = model_module.LSTM(input_size=(input_size_ex, 1), output_size=output_size, 
                             lstm_hidden_dim=lstm_hidden_dim, lstm_n_layer=lstm_n_layer,
                             lstm_squeeze=False,  # 마지막 hidden만 사용
                             bidirectional=bidirectional, fc_hidden_list=fc_hidden_list,
                             dropout_p=dropout_p, batch_norm=batch_norm)
model_comb = model_module.LSTM(input_size=(input_size_comb, 2), output_size=output_size,
                               lstm_hidden_dim=lstm_hidden_dim, lstm_n_layer=lstm_n_layer,
                               lstm_squeeze=True, # 모든 hidden 사용
                               bidirectional=bidirectional, fc_hidden_list=fc_hidden_list,
                               dropout_p=dropout_p, batch_norm=batch_norm)

print(f'model_ex: \n{model_ex}')
print(f'\nmodel_comb: \n{model_comb}')

model_ex: 
LSTMModel(
  (lstm): LSTM(1, 192, batch_first=True)
  (fc): MLP_Base(
    (layers): Sequential(
      (linear_0): Linear(in_features=192, out_features=1, bias=True)
    )
  )
)
Trainable parameters: 149953

model_comb: 
LSTMModel(
  (lstm): LSTM(2, 192, batch_first=True)
  (fc): MLP_Base(
    (layers): Sequential(
      (linear_0): Linear(in_features=4608, out_features=1, bias=True)
    )
  )
)
Trainable parameters: 155137


In [10]:
from trainer import train_val

args = {
    'dataset': {'args' : {'in_columns': in_cols_ex, 'out_columns': out_cols, 'input_days': input_days, 'time_lag': time_lag, 'scaling': scaling}},
    'model': 'LSTM',
    'device': 'cpu',
    'wandb': False,
    'loss': 'MSELoss',
    'metrics': ['MAELoss'],
    'optimizer': {'type': 'Adam', 'args': {'lr': 1e-3}},
    'lr_scheduler': {'enable': True, 'type': 'ReduceLROnPlateau', 'args': {'factor': 0.1, 'patience': 5}},
    'train_config': {'epochs': 20, 'save_period': 5, 'save_best_model': True, 'print_period': 1, 'save_result': True},
    'save_dir': {'checkpoint': 'saved/checkpoint'},
    'timestamp' : 'toy'
}

from omegaconf import OmegaConf
args = OmegaConf.create(args)

from utils import Logger
log_path = 'saved/log/test'
os.makedirs(log_path, exist_ok=True)
logger = Logger(args, log_path)
logger.save_args()
os.makedirs('saved/checkpoint', exist_ok=True)
os.makedirs('saved/result', exist_ok=True)


## exchange 학습  (1일 후의 exchange 예측)

In [11]:
train_pred_ex, test_pred_ex = train_val(args, logger, model_ex, train_loader_ex, test_loader_ex)

[Epoch 01] TRAIN LOSS(MSE) : 0.363 | TRAIN MAE : 0.332 | VALID MSE : 0.272 | VALID MAE : 0.418
[Epoch 01] Save Model in saved/checkpoint/toy_LSTM-best.pt
[Epoch 02] TRAIN LOSS(MSE) : 0.157 | TRAIN MAE : 0.264 | VALID MSE : 0.197 | VALID MAE : 0.358
[Epoch 02] Save Model in saved/checkpoint/toy_LSTM-best.pt
[Epoch 03] TRAIN LOSS(MSE) : 0.110 | TRAIN MAE : 0.233 | VALID MSE : 0.188 | VALID MAE : 0.350
[Epoch 03] Save Model in saved/checkpoint/toy_LSTM-best.pt
[Epoch 04] TRAIN LOSS(MSE) : 0.111 | TRAIN MAE : 0.262 | VALID MSE : 0.323 | VALID MAE : 0.485
[Epoch 05] TRAIN LOSS(MSE) : 0.087 | TRAIN MAE : 0.225 | VALID MSE : 0.143 | VALID MAE : 0.297
[Epoch 05] Save Model in saved/checkpoint/toy_LSTM-best.pt
[Epoch 06] TRAIN LOSS(MSE) : 0.089 | TRAIN MAE : 0.225 | VALID MSE : 0.213 | VALID MAE : 0.381
[Epoch 07] TRAIN LOSS(MSE) : 0.091 | TRAIN MAE : 0.211 | VALID MSE : 0.141 | VALID MAE : 0.299
[Epoch 07] Save Model in saved/checkpoint/toy_LSTM-best.pt
[Epoch 08] TRAIN LOSS(MSE) : 0.084 | TRA

In [12]:
from utils import visualize_result, make_result_df
results_ex = []
results_ex.append(make_result_df(args, 'TRAIN', 'train', train_ex, train_pred_ex))
results_ex.append(make_result_df(args, 'TEST', 'test', test_ex, test_pred_ex))

results_ex = pd.concat(results_ex, axis=0)
pd.DataFrame(results_ex).to_excel('toy_ex.xlsx')
display(results_ex.head())

fig_ex = visualize_result(results_ex)
fig_ex.write_html('fig_toy_ex.html')


mode            target      pred              input  \
            phase dataset    test_X    test_X test_X(24d before)   
date                                                               
2024-02-05  TRAIN   train -1.032814 -1.194787          -2.230651   
2024-02-06  TRAIN   train -1.158903 -1.096521          -2.057280   
2024-02-07  TRAIN   train -1.151022 -1.143842          -1.852387   
2024-02-08  TRAIN   train -1.135261 -1.149050          -1.639613   
2024-02-13  TRAIN   train -1.139201 -1.144563          -1.615972   

                                                                     \
           test_X(23d before) test_X(22d before) test_X(21d before)   
date                                                                  
2024-02-05          -2.057280          -1.852387          -1.639613   
2024-02-06          -1.852387          -1.639613          -1.615972   
2024-02-07          -1.639613          -1.615972          -1.627793   
2024-02-08          -1.615972          -1.627793          -1.454421   
2024-02-13          -1.627793          -1.454421          -1.738120   

                                                  ...                     \
           test_X(20d before) test_X(19d before)  ... test_X(10d before)   
date                                              ...                      
2024-02-05          -1.615972          -1.627793  ...          -0.713654   
2024-02-06          -1.627793          -1.454421  ...          -0.930368   
2024-02-07          -1.454421          -1.738120  ...          -0.788519   
2024-02-08          -1.738120          -1.714478  ...          -0.835802   
2024-02-13          -1.714478          -1.450481  ...          -0.816100   

                                                                  \
           test_X(9d before) test_X(8d before) test_X(7d before)   
date                                                               
2024-02-05         -0.930368         -0.788519         -0.835802   
2024-02-06         -0.788519         -0.835802         -0.816100   
2024-02-07         -0.835802         -0.816100         -0.839742   
2024-02-08         -0.816100         -0.839742         -1.087978   
2024-02-13         -0.839742         -1.087978         -0.883085   

                                                                  \
           test_X(6d before) test_X(5d before) test_X(4d before)   
date                                                               
2024-02-05         -0.816100         -0.839742         -1.087978   
2024-02-06         -0.839742         -1.087978         -0.883085   
2024-02-07         -1.087978         -0.883085         -0.993412   
2024-02-08         -0.883085         -0.993412         -1.355915   
2024-02-13         -0.993412         -1.355915         -1.032814   

                                                                  
           test_X(3d before) test_X(2d before) test_X(1d before)  
date                                                              
2024-02-05         -0.883085         -0.993412         -1.355915  
2024-02-06         -0.993412         -1.355915         -1.032814  
2024-02-07         -1.355915         -1.032814         -1.158902  
2024-02-08         -1.032814         -1.158902         -1.151022  
2024-02-13         -1.158902         -1.151022         -1.135261  

[5 rows x 28 columns]

## gold+exchange 학습 
- e.g. 13일 전의 gold와 1일 전의 exchange로 그 날의 gold 예측

In [13]:
train_pred_comb, test_pred_comb = train_val(args, logger, model_comb, train_loader_comb, test_loader_comb)

[Epoch 01] TRAIN LOSS(MSE) : 0.541 | TRAIN MAE : 0.442 | VALID MSE : 1.264 | VALID MAE : 1.024
[Epoch 01] Save Model in saved/checkpoint/toy_LSTM-best.pt
[Epoch 02] TRAIN LOSS(MSE) : 0.265 | TRAIN MAE : 0.350 | VALID MSE : 2.090 | VALID MAE : 1.328
[Epoch 03] TRAIN LOSS(MSE) : 0.136 | TRAIN MAE : 0.264 | VALID MSE : 2.646 | VALID MAE : 1.484
[Epoch 04] TRAIN LOSS(MSE) : 0.105 | TRAIN MAE : 0.208 | VALID MSE : 2.540 | VALID MAE : 1.466
[Epoch 05] TRAIN LOSS(MSE) : 0.079 | TRAIN MAE : 0.207 | VALID MSE : 2.498 | VALID MAE : 1.455
[Epoch 06] TRAIN LOSS(MSE) : 0.082 | TRAIN MAE : 0.220 | VALID MSE : 2.821 | VALID MAE : 1.541
[Epoch 07] TRAIN LOSS(MSE) : 0.095 | TRAIN MAE : 0.171 | VALID MSE : 2.931 | VALID MAE : 1.558
[Epoch 08] TRAIN LOSS(MSE) : 0.054 | TRAIN MAE : 0.180 | VALID MSE : 2.839 | VALID MAE : 1.538
[Epoch 09] TRAIN LOSS(MSE) : 0.055 | TRAIN MAE : 0.161 | VALID MSE : 2.996 | VALID MAE : 1.581
[Epoch 10] TRAIN LOSS(MSE) : 0.049 | TRAIN MAE : 0.160 | VALID MSE : 2.958 | VALID MAE

In [14]:
from utils import visualize_result, make_result_df

args['dataset']['args']['in_columns'] = in_cols_comb
results_comb = []
results_comb.append(make_result_df(args, 'TRAIN', 'train', train_comb, train_pred_comb))
results_comb.append(make_result_df(args, 'TEST', 'test', test_comb, test_pred_comb))

results_comb = pd.concat(results_comb, axis=0)
pd.DataFrame(results_comb).to_excel('toy_comb.xlsx')
display(results_comb.head())

fig_combine = visualize_result(results_comb)


fig_combine.write_html('fig_toy_comb.html')


mode            target      pred              input  \
            phase dataset    test_X    test_X test_Y(36d before)   
date                                                               
2024-02-05  TRAIN   train -1.032814 -1.025717          -1.538369   
2024-02-06  TRAIN   train -1.158903 -1.089045          -1.538369   
2024-02-07  TRAIN   train -1.151022 -1.161882          -1.538369   
2024-02-08  TRAIN   train -1.135261 -1.175782          -1.538369   
2024-02-13  TRAIN   train -1.139201 -1.210568          -1.538369   

                                                                     \
           test_Y(35d before) test_Y(34d before) test_Y(33d before)   
date                                                                  
2024-02-05          -1.538369          -1.538369          -1.538369   
2024-02-06          -1.538369          -1.538369          -1.538369   
2024-02-07          -1.538369          -1.538369          -1.538369   
2024-02-08          -1.538369          -1.538369          -1.538369   
2024-02-13          -1.538369          -1.538369          -1.538369   

                                                  ...                     \
           test_Y(32d before) test_Y(31d before)  ... test_X(10d before)   
date                                              ...                      
2024-02-05          -1.538369          -1.538369  ...          -0.713654   
2024-02-06          -1.538369          -1.538369  ...          -0.930368   
2024-02-07          -1.538369          -1.538369  ...          -0.788519   
2024-02-08          -1.538369          -1.538369  ...          -0.835802   
2024-02-13          -1.538369          -1.538369  ...          -0.816100   

                                                                  \
           test_X(9d before) test_X(8d before) test_X(7d before)   
date                                                               
2024-02-05         -0.930368         -0.788519         -0.835802   
2024-02-06         -0.788519         -0.835802         -0.816100   
2024-02-07         -0.835802         -0.816100         -0.839742   
2024-02-08         -0.816100         -0.839742         -1.087978   
2024-02-13         -0.839742         -1.087978         -0.883085   

                                                                  \
           test_X(6d before) test_X(5d before) test_X(4d before)   
date                                                               
2024-02-05         -0.816100         -0.839742         -1.087978   
2024-02-06         -0.839742         -1.087978         -0.883085   
2024-02-07         -1.087978         -0.883085         -0.993412   
2024-02-08         -0.883085         -0.993412         -1.355915   
2024-02-13         -0.993412         -1.355915         -1.032814   

                                                                  
           test_X(3d before) test_X(2d before) test_X(1d before)  
date                                                              
2024-02-05         -0.883085         -0.993412         -1.355915  
2024-02-06         -0.993412         -1.355915         -1.032814  
2024-02-07         -1.355915         -1.032814         -1.158902  
2024-02-08         -1.032814         -1.158902         -1.151022  
2024-02-13         -1.158902         -1.151022         -1.135261  

[5 rows x 52 columns]

In [15]:
# 두 시각화 합치기

import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy

fig_data = []
fig_ex_data = deepcopy(fig_ex.data)
for fig in fig_ex_data:
    if '(pred)' in fig.name:
        fig.name = fig.name.replace('(pred)', '(pred_ex)')
        fig.legendgroup = fig.name.split(' (')[0] + '_ex'
        fig.line.color = fig.line.color.replace('rgb', 'rgba').replace(')', ', 0.8)')
        fig_data.append(fig)
    elif '(target)' in fig.name:
        fig_data.append(fig)

fig_comb_data = deepcopy(fig_combine.data)
for fig in fig_comb_data:
    if '(pred)' in fig.name:
        fig.name = fig.name.replace('(pred)', '(pred_comb)')
        fig.legendgroup = fig.name.split(' (')[0] + '_comb'
        fig_data.append(fig)


# add background color to show train/test split
fig = go.Figure(fig_data, layout=fig_ex.layout)

fig.add_vrect(x0=test_ex.y_index[0], 
              x1=test_ex.y_index[-1], 
              fillcolor="bisque", 
              opacity=0.7, 
              layer="below", 
              line_width=0)



fig.show()

fig.write_html('fig_toy_final.html')

# 주황색 부분이 테스트 데이터에 해당함
